# Setup and Configuration for Streaming Feature Engineering Pipeline

This notebook handles the initial setup and configuration for the streaming feature engineering pipeline with Lakebase. It installs required packages, configures the environment, and validates the setup.

## Setup Tasks
1. **Package Installation**: Install required Python packages
2. **Environment Configuration**: Set up Spark configurations for streaming and Delta Lake
3. **Database Setup**: Create necessary databases and feature tables
4. **Validation**: Test all components and connections
5. **Sample Data**: Create initial sample datasets

## Prerequisites
- Databricks Runtime 17.3+ 
- Install Databricks Python SDK 0.65.0 or above on the cluster to support lakebase APIs
- Access to Lakebase Postgres Database

## Post-Setup
After running this notebook, you can proceed with:
- `01_fraud_detection_streaming.ipynb` - Streaming feature engineering pipeline to walkthrough real-time feature engineering and publish the features to Lakebase


In [0]:
# Import required libraries
# Note: 'spark' session is already available in Databricks
from pyspark.sql import Window
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable
import logging
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [0]:

# Import Lakebase client
from utils.lakebase_client import LakebaseClient

# Get OAuth token for authentication
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

# OR use secrets (recommended for production)
# token = dbutils.secrets.get(scope="lakebase", key="token")
# host = dbutils.secrets.get(scope="lakebase", key="host")

# Lakebase connection configuration
# TODO: Update with your actual Lakebase host
LAKEBASE_CONFIG = {
    "instance_name": "neha-lakebase-demo",
    "database": "databricks_postgres",    
}

print(" Connecting to Lakebase PostgreSQL...\n")

# Initialize Lakebase client
lakebase = LakebaseClient(**LAKEBASE_CONFIG)

# Test connection
print(" Testing Lakebase connection...")
if lakebase.test_connection():
    print(" Successfully connected to Lakebase PostgreSQL!")    
else:
    print(" Failed to connect to Lakebase")
    print("   Please check:")
    print("   1. Lakebase instance is provisioned")
    print("   2. Host is correct")
    print("   3. OAuth token is valid")
    raise Exception("Lakebase connection failed")

# Create feature table
print("\n Creating transaction_features table in Lakebase...")
lakebase.create_feature_table("transaction_features")
print(" Table created successfully!")

# Verify table exists
print("\n Verifying table...")
try:
    stats = lakebase.get_table_stats("transaction_features")
    print(f"   Total rows: {stats['total_rows']:,}")
    print(f"   Unique users: {stats['unique_users']:,}")
    print(f"   Unique merchants: {stats['unique_merchants']:,}")
except Exception as e:
    print("   Table exists but is empty (just created)")

print("\n" + "="*60)
print(" LAKEBASE POSTGRESQL SETUP COMPLETE")